# Text Data Processing with PyTorch

[Original tutorial](https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-i-5da6f1c89d84)

In [1]:
import os
import re
import spacy
import torch

import numpy as np
import pandas as pd

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import (
    Dataset
    , DataLoader
    , random_split
)

sys.path.insert(0, ".")
from load import *

In [2]:
# This is a small subset of the full toxic comment dataset
df_sample = pd.read_csv("./sample.csv", index_col="id")
df_sample.head(5)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
x = ToxicComtDataset("sample.csv")

In [4]:
v = x.get_vocab()

In [12]:
v.text_to_num("How do you do?")

[210, 24, 11, 24, 26]

In [15]:
v.vocab.itos[26]

'?'